In [1]:
 ! nvidia-smi

Mon Jun 20 12:42:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    46W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# ! gdown --id 1PUSJ56k93B42XW9oszcPDefM8HeJ2BOg
# ! pip -q uninstall -y kaggle
# ! pip -q install --upgrade pip
# ! pip -q install kaggle --upgrade
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle competitions download us-patent-phrase-to-phrase-matching
# ! kaggle datasets download yasufuminakama/cpc-data

In [3]:
# ! unzip -q /content/us-patent-phrase-to-phrase-matching.zip -d data
# ! rm /content/us-patent-phrase-to-phrase-matching.zip
# ! unzip -q /content/cpc-data.zip -d cpc_data
# ! rm /content/cpc-data.zip
# ! unzip /content/drive/MyDrive/USPPM-deberta-large-mnli-BCC-MSE-1-175-tags.zip -d weights

In [4]:
# ! pip -q install sentencepiece
# ! pip -q install transformers --upgrade
# ! pip -q install tokenizers --upgrade

In [5]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
print(f"torch.__version__: {torch.__version__}")
from torch.cuda import amp
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers

from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.__version__: 1.11.0+cu113
env: TOKENIZERS_PARALLELISM=true


In [6]:
# ====================================================
# CFG
# ====================================================
class CFG:
    wandb=True
    competition='PPPM'
    debug=False
    apex=True
    num_workers=4
    model="microsoft/deberta-v3-large"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=1
    num_warmup_steps=1000
    epochs=5
    encoder_lr=8e-5
    decoder_lr=8e-5
    min_lr=1e-7
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size = 64
    fc_dropout=0.0
    loss1 = "BCC"
    loss2 = "MSE"
    target_size=1
    max_len=512
    weight_decay=0.01
    n_accum=1
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    test_fold = 5
    train=True
    MV=1
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0, 1, 2, 3, 4]

In [7]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=CFG.seed)

In [8]:
def get_cpc_texts():
    contexts = []
    pattern = '[A-Z]\d+'
    for file_name in os.listdir('/content/cpc_data/CPCSchemeXML202105'):
        result = re.findall(pattern, file_name)
        if result:
            contexts.append(result)
    contexts = sorted(set(sum(contexts, [])))
    results = {}
    for cpc in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']:
        with open(f'/content/cpc_data/CPCTitleList202202/cpc-section-{cpc}_20220201.txt') as f:
            s = f.read()
        pattern = f'{cpc}\t\t.+'
        result = re.findall(pattern, s)
        cpc_result = result[0].lstrip(pattern)
        for context in [c for c in contexts if c[0] == cpc]:
            pattern = f'{context}\t\t.+'
            result = re.findall(pattern, s)
            results[context] = cpc_result + ". " + result[0].lstrip(pattern)
    return results

In [9]:
def get_folds(df):
    df['score_map'] = df['score'].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})
    encoder = LabelEncoder()
    df['anchor_map'] = encoder.fit_transform(df['anchor'])
    Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
    for n, (df_index, val_index) in enumerate(Fold.split(df, df['score_map'], groups = df['anchor_map'])):
        df.loc[val_index, 'fold'] = int(n)
    df['fold'] = df['fold'].astype(int)
    return df

In [10]:
def get_max_len(cpc_texts, train):
    lengths_dict = {}

    lengths = []
    tk0 = tqdm(cpc_texts.values(), total=len(cpc_texts))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        lengths.append(length)
    lengths_dict['context_text'] = lengths

    for text_col in ['anchor', 'target']:
        lengths = []
        tk0 = tqdm(train[text_col].fillna("").values, total=len(train))
        for text in tk0:
            length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
            lengths.append(length)
        lengths_dict[text_col] = lengths
        
    CFG.max_len = max(lengths_dict['anchor']) + max(lengths_dict['target'])\
                    + max(lengths_dict['context_text']) + 4 # CLS + SEP + SEP + SEP

In [11]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class usppmDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values
        self.labels = df['score'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float)
        return {"inputs" : inputs, 
                "labels" : label}

In [12]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            # self.model = AutoModel.from_pretrained("cfg.model", config=self.config)
            self.model = AutoModel.from_pretrained("/content/drive/MyDrive/pretrained_models/microsoft/deberta-v3-large-pretrain/checkpoint-23500/")
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def _resize_token_embeddings(self, length):
        self.model.resize_token_embeddings(length)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [13]:
def scoring(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score

In [14]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, outputs, targets):
        x = outputs - outputs.mean()
        y = targets - targets.mean()
        first = x / (torch.linalg.norm(x) + CFG.eps)
        second = y / (torch.linalg.norm(y) + CFG.eps)
        r = (first * second).sum()
        return -r

In [15]:
def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
            'lr': encoder_lr, 'weight_decay': weight_decay},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
            'lr': encoder_lr, 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if "model" not in n],
            'lr': decoder_lr, 'weight_decay': 0.0}
    ]
    return optimizer_parameters

In [16]:
def get_scheduler(cfg, optimizer, num_train_steps):
    if cfg.scheduler == 'linear':
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
        )
    elif cfg.scheduler == 'cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_warmup_steps= CFG.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles, last_epoch = -1
        )
    return scheduler

In [17]:
class Trainer:
    def __init__(self, config, dataloaders, optimizer, model, criterions, scheduler, device=CFG.device, apex=CFG.apex):
        self.train_loader, self.valid_loader = dataloaders
        self.criterion1, self.criterion2 = criterions
        self.scheduler = scheduler
        self.optimizer = optimizer
        self.model = model
        self.device = device
        self.apex = apex
        self.Config = config
    
    def train_one_epoch(self):
        scaler = amp.GradScaler()

        self.model.train()
        train_pbar = tqdm(enumerate(self.train_loader), total=len(self.train_loader))
        dataset_size = 0
        running_loss = 0.0

        for step, data in train_pbar:        
            for k, v in data["inputs"].items():
                data["inputs"][k] = v.to(self.device)

            data["labels"] = data["labels"].to(self.device)
            batch_size = data["labels"].size(0)

            with(amp.autocast(enabled = self.apex)):
                preds = self.model(data["inputs"])
                
                loss1 = self.criterion1(preds.view(-1,1), data["labels"].view(-1,1))
                loss2 = self.criterion2(preds.view(-1,1), data["labels"].view(-1,1))
                loss = (loss1 + loss2)/2
                loss /= self.Config.n_accum

            scaler.scale(loss).backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), self.Config.max_grad_norm)

            if (step + 1) % self.Config.n_accum == 0:
                    scaler.step(self.optimizer)
                    scaler.update()
                    self.optimizer.zero_grad()
                    self.scheduler.step()


            running_loss += (loss.item() * batch_size)  
            dataset_size += batch_size
            epoch_loss = running_loss / dataset_size
            mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0

            train_pbar.set_postfix(Train_Loss = epoch_loss, LR = self.optimizer.param_groups[0]['lr'], GPU_mem = f"{mem:.02f} GB")
        torch.cuda.empty_cache()
        gc.collect()
        return epoch_loss

    @torch.no_grad()
    def valid_one_epoch(self):

        self.model.eval()
        valid_pbar = tqdm(enumerate(self.valid_loader), total=len(self.valid_loader))

        dataset_size = 0
        running_loss = 0.0
        val_preds = []

        for step , data in valid_pbar:
            
            for k, v in data["inputs"].items():
                data["inputs"][k] = v.to(self.device)

            data["labels"] = data["labels"].to(self.device)
            batch_size = data["labels"].size(0)

            preds = self.model(data["inputs"])

            loss1 = self.criterion1(preds.view(-1,1), data["labels"].view(-1,1))
            loss2 = self.criterion2(preds.view(-1,1), data["labels"].view(-1,1))
            loss = (loss1 + loss2)/2
            
            val_preds.append(preds.sigmoid().to("cpu").numpy())
            running_loss += (loss.item()*batch_size)
            dataset_size += batch_size
        
            epoch_loss = running_loss / dataset_size


            mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
            valid_pbar.set_postfix(Valid_loss = epoch_loss,
                                   LR = self.optimizer.param_groups[0]['lr'],
                                   GPU_mem = f"{mem:.02f} GB" )
            
        val_preds  = np.concatenate(np.concatenate(val_preds))
        torch.cuda.empty_cache()
        gc.collect()
        return val_preds, epoch_loss   


    def fit(self, valid_labels, fold: str, epochs: int = 10, output_dir: str = "/content/models/", custom_name: str = 'model.pth',):
        """
        Low-effort alternative for doing the complete training and validation process
        """
        custom_name = f'model_{fold}.pth'
        best_score = int(-1e+7)
        oof_df = pd.DataFrame()
        for epx in range(epochs):
            print(f"{'='*20} Epoch: {epx+1} / {epochs} {'='*20}")

            train_loss = self.train_one_epoch()
            print(f"Training loss: {train_loss:.4f}")

            valid_preds, val_loss = self.valid_one_epoch()
            val_score = scoring(valid_labels, valid_preds)
            
            print(f'Validation Score: {val_score:.4f}')
            
            if val_score > best_score:
                print(f"Valid Score Improved ({best_score:0.4f} ---> {val_score:0.4f})")
                best_score = val_score
                self.save_model(output_dir, custom_name)
                print(f"Saved model with val_score: {best_score:.4f}")
                oof_df["preds"] = valid_preds
                oof_df["labels"] = valid_labels

            print(f"Best score is {best_score:0.5f}")
        
        oof_df.to_csv(f"models/oof_{fold}.csv", index = None) 

            

    def save_model(self, path, name, verbose=False):
        """
        Saves the model at the provided destination
        """
        
        try:
            if not os.path.exists(path):
                os.makedirs(path)
        except:
            print("Errors encountered while making the output directory")

        torch.save(self.model.state_dict(), os.path.join(path, name))
        if verbose:
            print(f"Model Saved at: {os.path.join(path, name)}")

In [18]:
if __name__ == "__main__":
    train = pd.read_csv("/content/data/train.csv")
    cpc_texts = get_cpc_texts()
    torch.save(cpc_texts, "cpc_texts.pth")
    train['context_text'] = train['context'].map(cpc_texts)
    train["context_tags"] = '<' + train['context'] + '>'
    context_tags = list(train['context_tags'].unique())
    train['text'] = train['context_tags'] + '<anchor>' + train['anchor'] + '</anchor><target>' + train['target'] + '</target><context>'  + train['context_text'] + '</context>'
    train["text"] = train["text"].map(lambda x: x.lower())
    if CFG.debug:
        train = train.sample(n=1000, random_state=0).reset_index(drop=True)
    train = get_folds(train)
    
    tokenizer = AutoTokenizer.from_pretrained(CFG.model)
    tokenizer.add_special_tokens({'additional_special_tokens': ['<anchor>', '</anchor>', '<target>', '</target>', '<context>', '</context>'] + context_tags})
    CFG.tokenizer = tokenizer
    get_max_len(cpc_texts, train)

    criterion1 = nn.BCEWithLogitsLoss(reduction="mean")
    criterion2 = nn.MSELoss()
    for fold in CFG.trn_fold:
        print("*" * 20)
        print(f"Training fold {fold}")
        print("*" * 20)
        val_df = train[train.fold == fold].reset_index(drop = True)
        train_df = train[(train.fold != fold)].reset_index(drop = True)

        train_dataset = usppmDataset(CFG, train_df)
        val_dataset = usppmDataset(CFG, val_df)

        train_loader = DataLoader(train_dataset,
                                batch_size=CFG.batch_size,
                                shuffle=True,
                                num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
        valid_loader = DataLoader(val_dataset,
                                batch_size=CFG.batch_size,
                                shuffle=False,
                                num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
        
        
        model = CustomModel(CFG, config_path=None, pretrained=True)
        model._resize_token_embeddings(len(CFG.tokenizer))
        model.to(device)

        optimizer_parameters = get_optimizer_params(model,
                                                    encoder_lr=CFG.encoder_lr, 
                                                    decoder_lr=CFG.decoder_lr,
                                                    weight_decay=CFG.weight_decay)
        optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)

        num_train_steps = int(len(train) / CFG.batch_size * CFG.epochs)
        scheduler = get_scheduler(CFG, optimizer, num_train_steps)

        trainer = Trainer(config = CFG,
                          dataloaders = (train_loader, valid_loader),
                          optimizer = optimizer,
                          model = model,
                          criterions = (criterion1, criterion2),
                          scheduler = scheduler)
        
        trainer.fit(valid_labels = val_df["score"].values,
                    epochs = CFG.epochs,
                    fold = fold,
                    output_dir = f"/content/models/")

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/136 [00:00<?, ?it/s]

...

Validation Score: 0.8545
Valid Score Improved (0.8414 ---> 0.8545)
Saved model with val_score: 0.8545
Best score is 0.85452
==================== Epoch: 5 / 5 ====================


  0%|          | 0/455 [00:00<?, ?it/s]

Training loss: 0.3489


  0%|          | 0/114 [00:00<?, ?it/s]

Validation Score: 0.8527
Best score is 0.85452


In [19]:
! mkdir /content/models/tokenizer
CFG.tokenizer.save_pretrained("/content/models/tokenizer/")

('/content/models/tokenizer/tokenizer_config.json',
 '/content/models/tokenizer/special_tokens_map.json',
 '/content/models/tokenizer/spm.model',
 '/content/models/tokenizer/added_tokens.json',
 '/content/models/tokenizer/tokenizer.json')

In [20]:
model = CFG.model.split("/")[-1]
save_file = f"/content/drive/MyDrive/USPPM-{model}-PT-{CFG.loss1}-{CFG.loss2}-{CFG.MV}-{CFG.max_len}-tags.zip"
! zip -r $save_file  models

  adding: models/ (stored 0%)
  adding: models/model_0.pth (deflated 7%)
  adding: models/oof_4.csv (deflated 67%)
  adding: models/model_2.pth (deflated 7%)
  adding: models/oof_3.csv (deflated 67%)
  adding: models/model_3.pth (deflated 7%)
  adding: models/oof_0.csv (deflated 67%)
  adding: models/tokenizer/ (stored 0%)
  adding: models/tokenizer/tokenizer.json (deflated 77%)
  adding: models/tokenizer/special_tokens_map.json (deflated 75%)
  adding: models/tokenizer/added_tokens.json (deflated 76%)
  adding: models/tokenizer/tokenizer_config.json (deflated 45%)
  adding: models/tokenizer/spm.model (deflated 50%)
  adding: models/model_4.pth (deflated 7%)
  adding: models/oof_2.csv (deflated 67%)
  adding: models/model_1.pth (deflated 7%)
  adding: models/oof_1.csv (deflated 67%)
